## Ενσωματώσεις

Στο προηγούμενο παράδειγμά μας, εργαστήκαμε με διανύσματα υψηλής διάστασης τύπου bag-of-words με μήκος `vocab_size`, και μετατρέψαμε ρητά διανύσματα χαμηλής διάστασης θέσης σε αραιή αναπαράσταση one-hot. Αυτή η αναπαράσταση one-hot δεν είναι αποδοτική από άποψη μνήμης. Επιπλέον, κάθε λέξη αντιμετωπίζεται ανεξάρτητα από τις άλλες, οπότε τα κωδικοποιημένα διανύσματα one-hot δεν εκφράζουν τις σημασιολογικές ομοιότητες μεταξύ των λέξεων.

Σε αυτήν την ενότητα, θα συνεχίσουμε να εξερευνούμε το σύνολο δεδομένων **News AG**. Για να ξεκινήσουμε, ας φορτώσουμε τα δεδομένα και ας πάρουμε κάποιους ορισμούς από την προηγούμενη ενότητα.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### Τι είναι ένα embedding;

Η ιδέα του **embedding** είναι να αναπαραστήσουμε λέξεις χρησιμοποιώντας χαμηλότερης διάστασης πυκνά διανύσματα που αντανακλούν τη σημασιολογική έννοια της λέξης. Αργότερα θα συζητήσουμε πώς να δημιουργήσουμε ουσιαστικά word embeddings, αλλά προς το παρόν ας σκεφτούμε τα embeddings ως έναν τρόπο να μειώσουμε τη διαστατικότητα ενός διανύσματος λέξης.

Έτσι, ένα embedding layer παίρνει μια λέξη ως είσοδο και παράγει ένα διανύσμα εξόδου με καθορισμένο `embedding_size`. Με μια έννοια, είναι πολύ παρόμοιο με ένα `Dense` layer, αλλά αντί να παίρνει ένα one-hot encoded διάνυσμα ως είσοδο, μπορεί να πάρει έναν αριθμό λέξης.

Χρησιμοποιώντας ένα embedding layer ως το πρώτο layer στο δίκτυό μας, μπορούμε να μεταβούμε από το μοντέλο bag-of-words σε ένα μοντέλο **embedding bag**, όπου πρώτα μετατρέπουμε κάθε λέξη στο κείμενό μας στο αντίστοιχο embedding και στη συνέχεια υπολογίζουμε κάποια συνάρτηση συσσωμάτωσης πάνω σε όλα αυτά τα embeddings, όπως `sum`, `average` ή `max`.

![Εικόνα που δείχνει έναν ταξινομητή embedding για πέντε λέξεις ακολουθίας.](../../../../../translated_images/el/embedding-classifier-example.b77f021a7ee67eee.webp)

Το νευρωνικό δίκτυο ταξινομητή μας αποτελείται από τα εξής layers:

* Το layer `TextVectorization`, το οποίο παίρνει μια συμβολοσειρά ως είσοδο και παράγει έναν tensor αριθμών token. Θα καθορίσουμε ένα λογικό μέγεθος λεξιλογίου `vocab_size` και θα αγνοήσουμε λέξεις που χρησιμοποιούνται λιγότερο συχνά. Το σχήμα εισόδου θα είναι 1, και το σχήμα εξόδου θα είναι $n$, καθώς θα πάρουμε $n$ tokens ως αποτέλεσμα, καθένα από τα οποία περιέχει αριθμούς από 0 έως `vocab_size`.
* Το layer `Embedding`, το οποίο παίρνει $n$ αριθμούς και μειώνει κάθε αριθμό σε ένα πυκνό διάνυσμα ενός δεδομένου μήκους (100 στο παράδειγμά μας). Έτσι, το tensor εισόδου με σχήμα $n$ θα μετατραπεί σε ένα tensor $n\times 100$.
* Το layer συσσωμάτωσης, το οποίο παίρνει τον μέσο όρο αυτού του tensor κατά τον πρώτο άξονα, δηλαδή θα υπολογίσει τον μέσο όρο όλων των $n$ tensors εισόδου που αντιστοιχούν σε διαφορετικές λέξεις. Για να υλοποιήσουμε αυτό το layer, θα χρησιμοποιήσουμε ένα layer `Lambda` και θα περάσουμε σε αυτό τη συνάρτηση για τον υπολογισμό του μέσου όρου. Η έξοδος θα έχει σχήμα 100 και θα είναι η αριθμητική αναπαράσταση ολόκληρης της ακολουθίας εισόδου.
* Τελικός γραμμικός ταξινομητής `Dense`.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


Στην εκτύπωση `summary`, στη στήλη **output shape**, η πρώτη διάσταση του tensor `None` αντιστοιχεί στο μέγεθος του minibatch, και η δεύτερη αντιστοιχεί στο μήκος της ακολουθίας των tokens. Όλες οι ακολουθίες tokens στο minibatch έχουν διαφορετικά μήκη. Θα συζητήσουμε πώς να το διαχειριστούμε στην επόμενη ενότητα.

Τώρα ας εκπαιδεύσουμε το δίκτυο:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Σημείωση** ότι δημιουργούμε τον vectorizer βασισμένο σε ένα υποσύνολο των δεδομένων. Αυτό γίνεται για να επιταχυνθεί η διαδικασία, και μπορεί να οδηγήσει σε μια κατάσταση όπου δεν υπάρχουν όλα τα tokens από το κείμενό μας στο λεξιλόγιο. Σε αυτή την περίπτωση, αυτά τα tokens θα αγνοηθούν, κάτι που μπορεί να οδηγήσει σε ελαφρώς χαμηλότερη ακρίβεια. Ωστόσο, στην πραγματική ζωή, ένα υποσύνολο κειμένου συχνά δίνει μια καλή εκτίμηση λεξιλογίου.


### Αντιμετώπιση διαφορετικών μεγεθών ακολουθιών μεταβλητών

Ας κατανοήσουμε πώς γίνεται η εκπαίδευση σε μικροπαρτίδες. Στο παραπάνω παράδειγμα, ο tensor εισόδου έχει διάσταση 1, και χρησιμοποιούμε μικροπαρτίδες μήκους 128, έτσι ώστε το πραγματικό μέγεθος του tensor να είναι $128 \times 1$. Ωστόσο, ο αριθμός των tokens σε κάθε πρόταση είναι διαφορετικός. Εάν εφαρμόσουμε το layer `TextVectorization` σε μία μόνο είσοδο, ο αριθμός των tokens που επιστρέφονται είναι διαφορετικός, ανάλογα με το πώς γίνεται η τοκενοποίηση του κειμένου:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Ωστόσο, όταν εφαρμόζουμε τον vectorizer σε αρκετές ακολουθίες, πρέπει να παράγει έναν τανυστή ορθογώνιου σχήματος, οπότε γεμίζει τα μη χρησιμοποιημένα στοιχεία με το PAD token (το οποίο στην περίπτωσή μας είναι μηδέν):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Εδώ μπορούμε να δούμε τις ενσωματώσεις:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Σημείωση**: Για να ελαχιστοποιηθεί η ποσότητα της συμπλήρωσης, σε ορισμένες περιπτώσεις έχει νόημα να ταξινομηθούν όλες οι ακολουθίες στο σύνολο δεδομένων με τη σειρά αύξησης του μήκους (ή, πιο συγκεκριμένα, του αριθμού των τοκεν). Αυτό θα διασφαλίσει ότι κάθε minibatch περιέχει ακολουθίες παρόμοιου μήκους.


## Σημασιολογικές ενσωματώσεις: Word2Vec

Στο προηγούμενο παράδειγμα, το επίπεδο ενσωμάτωσης έμαθε να αντιστοιχεί λέξεις σε διανυσματικές αναπαραστάσεις, ωστόσο αυτές οι αναπαραστάσεις δεν είχαν σημασιολογική έννοια. Θα ήταν χρήσιμο να μάθουμε μια διανυσματική αναπαράσταση όπου παρόμοιες λέξεις ή συνώνυμα αντιστοιχούν σε διανύσματα που βρίσκονται κοντά μεταξύ τους με βάση κάποια διανυσματική απόσταση (για παράδειγμα, την ευκλείδεια απόσταση).

Για να το πετύχουμε αυτό, πρέπει να προεκπαιδεύσουμε το μοντέλο ενσωμάτωσης σε μια μεγάλη συλλογή κειμένων χρησιμοποιώντας μια τεχνική όπως το [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Βασίζεται σε δύο κύριες αρχιτεκτονικές που χρησιμοποιούνται για την παραγωγή μιας κατανεμημένης αναπαράστασης των λέξεων:

 - **Συνεχές bag-of-words** (CBoW), όπου εκπαιδεύουμε το μοντέλο να προβλέπει μια λέξη από το περιβάλλον της. Δεδομένου του ngram $(W_{-2},W_{-1},W_0,W_1,W_2)$, ο στόχος του μοντέλου είναι να προβλέψει το $W_0$ από $(W_{-2},W_{-1},W_1,W_2)$.
 - **Συνεχές skip-gram**, το οποίο είναι αντίθετο του CBoW. Το μοντέλο χρησιμοποιεί το παράθυρο των λέξεων του περιβάλλοντος για να προβλέψει την τρέχουσα λέξη.

Το CBoW είναι πιο γρήγορο, ενώ το skip-gram, αν και πιο αργό, αποδίδει καλύτερα στην αναπαράσταση σπάνιων λέξεων.

![Εικόνα που δείχνει τους αλγόριθμους CBoW και Skip-Gram για τη μετατροπή λέξεων σε διανύσματα.](../../../../../translated_images/el/example-algorithms-for-converting-words-to-vectors.fbe9207a726922f6.webp)

Για να πειραματιστούμε με την ενσωμάτωση Word2Vec που έχει προεκπαιδευτεί στο σύνολο δεδομένων Google News, μπορούμε να χρησιμοποιήσουμε τη βιβλιοθήκη **gensim**. Παρακάτω βρίσκουμε τις λέξεις που είναι πιο παρόμοιες με τη λέξη 'neural'.

> **Σημείωση:** Όταν δημιουργείτε για πρώτη φορά διανύσματα λέξεων, η λήψη τους μπορεί να πάρει αρκετό χρόνο!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


Μπορούμε επίσης να εξαγάγουμε την ενσωμάτωση διανύσματος από τη λέξη, για να χρησιμοποιηθεί στην εκπαίδευση του μοντέλου ταξινόμησης. Η ενσωμάτωση έχει 300 συνιστώσες, αλλά εδώ δείχνουμε μόνο τις πρώτες 20 συνιστώσες του διανύσματος για λόγους σαφήνειας:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Το σπουδαίο με τις σημασιολογικές ενσωματώσεις είναι ότι μπορείτε να χειριστείτε την κωδικοποίηση του διανύσματος βάσει της σημασιολογίας. Για παράδειγμα, μπορούμε να ζητήσουμε να βρούμε μια λέξη της οποίας η διανυσματική αναπαράσταση είναι όσο το δυνατόν πιο κοντά στις λέξεις *βασιλιάς* και *γυναίκα*, και όσο το δυνατόν πιο μακριά από τη λέξη *άνδρας*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Ένα παράδειγμα παραπάνω χρησιμοποιεί κάποια εσωτερική μαγεία του GenSym, αλλά η υποκείμενη λογική είναι στην πραγματικότητα αρκετά απλή. Ένα ενδιαφέρον στοιχείο σχετικά με τα embeddings είναι ότι μπορείτε να εκτελέσετε κανονικές διανυσματικές πράξεις σε διανύσματα embeddings, και αυτό θα αντικατοπτρίζει πράξεις στις **σημασίες** των λέξεων. Το παραπάνω παράδειγμα μπορεί να εκφραστεί με όρους διανυσματικών πράξεων: υπολογίζουμε το διάνυσμα που αντιστοιχεί στο **ΒΑΣΙΛΙΑΣ-ΑΝΔΡΑΣ+ΓΥΝΑΙΚΑ** (οι πράξεις `+` και `-` εκτελούνται στις διανυσματικές αναπαραστάσεις των αντίστοιχων λέξεων), και στη συνέχεια βρίσκουμε τη λέξη στο λεξικό που είναι πιο κοντά σε αυτό το διάνυσμα:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **ΣΗΜΕΙΩΣΗ**: Χρειάστηκε να προσθέσουμε μικρούς συντελεστές στους διανύσματα *man* και *woman* - δοκιμάστε να τους αφαιρέσετε για να δείτε τι συμβαίνει.

Για να βρούμε το κοντινότερο διάνυσμα, χρησιμοποιούμε τη μηχανή του TensorFlow για να υπολογίσουμε ένα διάνυσμα αποστάσεων μεταξύ του δικού μας διανύσματος και όλων των διανυσμάτων στο λεξιλόγιο, και στη συνέχεια βρίσκουμε τον δείκτη της ελάχιστης λέξης χρησιμοποιώντας `argmin`.


Ενώ το Word2Vec φαίνεται να είναι ένας εξαιρετικός τρόπος για να εκφράσει κανείς τη σημασιολογία των λέξεων, έχει αρκετά μειονεκτήματα, όπως τα εξής:

* Τα μοντέλα CBoW και skip-gram είναι **προβλεπτικά embeddings**, και λαμβάνουν υπόψη μόνο το τοπικό πλαίσιο. Το Word2Vec δεν αξιοποιεί το παγκόσμιο πλαίσιο.
* Το Word2Vec δεν λαμβάνει υπόψη τη **μορφολογία** των λέξεων, δηλαδή το γεγονός ότι η σημασία μιας λέξης μπορεί να εξαρτάται από διαφορετικά μέρη της λέξης, όπως η ρίζα.

Το **FastText** προσπαθεί να ξεπεράσει τον δεύτερο περιορισμό και βασίζεται στο Word2Vec, μαθαίνοντας διανυσματικές αναπαραστάσεις για κάθε λέξη και για τα n-grams χαρακτήρων που βρίσκονται μέσα σε κάθε λέξη. Οι τιμές αυτών των αναπαραστάσεων στη συνέχεια υπολογίζονται κατά μέσο όρο σε ένα διάνυσμα σε κάθε βήμα εκπαίδευσης. Παρόλο που αυτό προσθέτει αρκετό επιπλέον υπολογιστικό φόρτο κατά την προεκπαίδευση, επιτρέπει στα embeddings λέξεων να κωδικοποιούν πληροφορίες υπολέξεων.

Μια άλλη μέθοδος, το **GloVe**, χρησιμοποιεί μια διαφορετική προσέγγιση για τα embeddings λέξεων, βασισμένη στη παραγοντοποίηση του πίνακα λέξεων-πλαισίων. Αρχικά, δημιουργεί έναν μεγάλο πίνακα που μετρά τον αριθμό εμφανίσεων λέξεων σε διαφορετικά πλαίσια και στη συνέχεια προσπαθεί να αναπαραστήσει αυτόν τον πίνακα σε χαμηλότερες διαστάσεις με τρόπο που να ελαχιστοποιεί την απώλεια ανακατασκευής.

Η βιβλιοθήκη gensim υποστηρίζει αυτά τα embeddings λέξεων, και μπορείτε να πειραματιστείτε μαζί τους αλλάζοντας τον κώδικα φόρτωσης του μοντέλου παραπάνω.


## Χρήση προεκπαιδευμένων ενσωματώσεων στο Keras

Μπορούμε να τροποποιήσουμε το παραπάνω παράδειγμα ώστε να προετοιμάσουμε τη μήτρα στο επίπεδο ενσωμάτωσης μας με σημασιολογικές ενσωματώσεις, όπως το Word2Vec. Τα λεξιλόγια της προεκπαιδευμένης ενσωμάτωσης και του κειμενικού σώματος πιθανότατα δεν θα ταιριάζουν, οπότε πρέπει να επιλέξουμε ένα. Εδώ εξετάζουμε τις δύο πιθανές επιλογές: τη χρήση του λεξιλογίου του tokenizer και τη χρήση του λεξιλογίου από τις ενσωματώσεις Word2Vec.

### Χρήση λεξιλογίου tokenizer

Όταν χρησιμοποιούμε το λεξιλόγιο του tokenizer, ορισμένες από τις λέξεις του λεξιλογίου θα έχουν αντίστοιχες ενσωματώσεις Word2Vec, ενώ κάποιες θα λείπουν. Δεδομένου ότι το μέγεθος του λεξιλογίου μας είναι `vocab_size`, και το μήκος του διανύσματος ενσωμάτωσης Word2Vec είναι `embed_size`, το επίπεδο ενσωμάτωσης θα αναπαρίσταται από μια μήτρα βαρών με σχήμα `vocab_size`$\times$`embed_size`. Θα γεμίσουμε αυτήν τη μήτρα περνώντας μέσα από το λεξιλόγιο:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Για λέξεις που δεν υπάρχουν στο λεξιλόγιο του Word2Vec, μπορούμε είτε να τις αφήσουμε ως μηδενικά, είτε να δημιουργήσουμε έναν τυχαίο διάνυσμα.

Τώρα μπορούμε να ορίσουμε ένα embedding layer με προκαθορισμένα βάρη:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Σημείωση**: Παρατηρήστε ότι ορίζουμε `trainable=False` κατά τη δημιουργία του `Embedding`, που σημαίνει ότι δεν επανεκπαιδεύουμε το Embedding layer. Αυτό μπορεί να προκαλέσει ελαφρώς χαμηλότερη ακρίβεια, αλλά επιταχύνει την εκπαίδευση.

### Χρήση λεξιλογίου ενσωμάτωσης

Ένα πρόβλημα με την προηγούμενη προσέγγιση είναι ότι τα λεξιλόγια που χρησιμοποιούνται στο TextVectorization και στο Embedding είναι διαφορετικά. Για να ξεπεράσουμε αυτό το πρόβλημα, μπορούμε να χρησιμοποιήσουμε μία από τις παρακάτω λύσεις:
* Επανεκπαίδευση του μοντέλου Word2Vec στο δικό μας λεξιλόγιο.
* Φόρτωση του dataset μας με το λεξιλόγιο από το προεκπαιδευμένο μοντέλο Word2Vec. Τα λεξιλόγια που χρησιμοποιούνται για τη φόρτωση του dataset μπορούν να καθοριστούν κατά τη φόρτωση.

Η δεύτερη προσέγγιση φαίνεται πιο εύκολη, οπότε ας την υλοποιήσουμε. Πρώτα απ' όλα, θα δημιουργήσουμε ένα `TextVectorization` layer με το καθορισμένο λεξιλόγιο, το οποίο προέρχεται από τις ενσωματώσεις Word2Vec:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

Η βιβλιοθήκη ενσωματώσεων λέξεων gensim περιέχει μια βολική συνάρτηση, `get_keras_embeddings`, η οποία θα δημιουργήσει αυτόματα το αντίστοιχο επίπεδο ενσωματώσεων Keras για εσάς.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Ένας από τους λόγους που δεν βλέπουμε μεγαλύτερη ακρίβεια είναι επειδή ορισμένες λέξεις από το σύνολο δεδομένων μας λείπουν από το προεκπαιδευμένο λεξιλόγιο του GloVe και, επομένως, ουσιαστικά αγνοούνται. Για να ξεπεράσουμε αυτό, μπορούμε να εκπαιδεύσουμε τις δικές μας ενσωματώσεις βασισμένες στο σύνολο δεδομένων μας.


## Ενσωματώσεις με βάση τα συμφραζόμενα

Ένας βασικός περιορισμός των παραδοσιακών προεκπαιδευμένων αναπαραστάσεων ενσωμάτωσης, όπως το Word2Vec, είναι το γεγονός ότι, παρόλο που μπορούν να αποτυπώσουν κάποια σημασία μιας λέξης, δεν μπορούν να διακρίνουν μεταξύ διαφορετικών νοημάτων. Αυτό μπορεί να προκαλέσει προβλήματα στα μοντέλα που χρησιμοποιούνται στη συνέχεια.

Για παράδειγμα, η λέξη «play» έχει διαφορετική σημασία στις παρακάτω δύο προτάσεις:
- Πήγα σε μια **παράσταση** στο θέατρο.
- Ο Τζον θέλει να **παίξει** με τους φίλους του.

Οι προεκπαιδευμένες ενσωματώσεις που αναφέραμε αναπαριστούν και τις δύο σημασίες της λέξης «play» με την ίδια ενσωμάτωση. Για να ξεπεράσουμε αυτόν τον περιορισμό, πρέπει να δημιουργήσουμε ενσωματώσεις βασισμένες στο **γλωσσικό μοντέλο**, το οποίο έχει εκπαιδευτεί σε ένα μεγάλο σώμα κειμένου και *γνωρίζει* πώς οι λέξεις μπορούν να συνδυαστούν σε διαφορετικά συμφραζόμενα. Η συζήτηση για τις ενσωματώσεις με βάση τα συμφραζόμενα ξεφεύγει από το πλαίσιο αυτού του μαθήματος, αλλά θα επανέλθουμε σε αυτές όταν μιλήσουμε για τα γλωσσικά μοντέλα στην επόμενη ενότητα.



---

**Αποποίηση ευθύνης**:  
Αυτό το έγγραφο έχει μεταφραστεί χρησιμοποιώντας την υπηρεσία αυτόματης μετάφρασης AI [Co-op Translator](https://github.com/Azure/co-op-translator). Παρόλο που καταβάλλουμε κάθε προσπάθεια για ακρίβεια, παρακαλούμε να έχετε υπόψη ότι οι αυτόματες μεταφράσεις ενδέχεται να περιέχουν σφάλματα ή ανακρίβειες. Το πρωτότυπο έγγραφο στη μητρική του γλώσσα θα πρέπει να θεωρείται η αυθεντική πηγή. Για κρίσιμες πληροφορίες, συνιστάται επαγγελματική ανθρώπινη μετάφραση. Δεν φέρουμε ευθύνη για τυχόν παρεξηγήσεις ή εσφαλμένες ερμηνείες που προκύπτουν από τη χρήση αυτής της μετάφρασης.
